In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/datasetFoodImage/images.zip" -d "/content/sample_data/data"

In [3]:
import json
import os
import tensorflow as tf
from google.colab import files
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras import Sequential, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback
from tensorflow_hub import KerasLayer
from tensorflow.keras.metrics import Precision,Recall
import math

In [4]:
model = tf.keras.models.load_model('/content/drive/MyDrive/models/EffnetWithAugmentDense+_05_0.63.h5', custom_objects={'KerasLayer': KerasLayer})

KeyboardInterrupt: ignored

In [ ]:
model.compile(optimizer='Adam', loss="categorical_crossentropy", metrics=["accuracy", Precision(), Recall()])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 EfficientNet (KerasLayer)   (None, 1280)              117746848 
                                                                 
 dense (Dense)               (None, 256)               327936    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 108)               27756     
                                                                 
Total params: 118,103,564
Trainable params: 356,204
Non-trainable params: 117,747,360
____________________________________

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=90,
    zoom_range=0.3,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    rescale = 1./255,
    validation_split=0.1)

In [ ]:
valdatagen = ImageDataGenerator(
    rescale = 1./255,
    validation_split=0.1)

In [ ]:
train_data_generator = datagen.flow_from_directory(
    "/content/sample_data/data/images",
    target_size=(250,250),
    seed=123,
    subset='training'
)

Found 91064 images belonging to 108 classes.


In [ ]:
valid_data_generator = valdatagen.flow_from_directory(
    "/content/sample_data/data/images",
    seed = 123,
    target_size=(250,250),
    subset='validation'
)

Found 10115 images belonging to 108 classes.


In [ ]:
checkpoint = ModelCheckpoint("/content/drive/MyDrive/models/EffnetWithAugmentDense+_{epoch:02d}_{accuracy:.2f}.h5", verbose=1, mode='auto', save_freq=2846)

In [ ]:
class myCallback(Callback):
  def on_epoch_end(self, epoch, logs={}):
    path = '/content/drive/MyDrive/models/trainingHistoryEfficientNetDense+.json'
    with open(path) as f:
      data = json.load(f)
    os.remove(path)
    dictLogs = {
        'loss': logs.get('loss'),
        'accuracy': logs.get('accuracy'),
        'val_loss': logs.get('val_loss'),
        'val_accuracy': logs.get('val_accuracy'),
        'val_precision': logs.get('val_precision'),
        'val_recall': logs.get('val_recall'),
        'recall': logs.get('recall'),
        'precision': logs.get('precision')
    }
    data.append(dictLogs)
    json_object = json.dumps(data, indent = 4)
    with open(path, "w") as outJson:
      outJson.write(json_object)

In [ ]:
callbacks = myCallback()

In [ ]:
model.fit(train_data_generator, validation_data=valid_data_generator, epochs=5,callbacks=[checkpoint, callbacks])

Epoch 1/5
2845/2846 [============================>.] - ETA: 0s - loss: 1.5054 - accuracy: 0.6140 - precision: 0.8249 - recall: 0.4845
Epoch 1: saving model to /content/drive/MyDrive/models/EffnetWithAugmentDense+_01_0.61.h5
2846/2846 [==============================] - 2243s 776ms/step - loss: 1.5055 - accuracy: 0.6140 - precision: 0.8249 - recall: 0.4845 - val_loss: 0.8795 - val_accuracy: 0.7658 - val_precision: 0.9096 - val_recall: 0.6478
Epoch 2/5
2845/2846 [============================>.] - ETA: 0s - loss: 1.4881 - accuracy: 0.6168 - precision: 0.8286 - recall: 0.4873
Epoch 2: saving model to /content/drive/MyDrive/models/EffnetWithAugmentDense+_02_0.62.h5
2846/2846 [==============================] - 2168s 762ms/step - loss: 1.4881 - accuracy: 0.6168 - precision: 0.8286 - recall: 0.4873 - val_loss: 0.8645 - val_accuracy: 0.7748 - val_precision: 0.9122 - val_recall: 0.6594
Epoch 3/5
2845/2846 [============================>.] - ETA: 0s - loss: 1.4550 - accuracy: 0.6235 - precision: 0.